# Examples of split-apply-combine in pandas

From

1. [pandas guide](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html)
2. [pandas cookbook](https://pandas.pydata.org/pandas-docs/stable/user_guide/cookbook.html#cookbook-grouping)


## Setup libraries

In [1]:
import pandas as pd
import numpy as np

## Splitting an object into groups

In [2]:
df = pd.DataFrame([('bird', 'Falconiformes', 389.0),
                   ('bird', 'Psittaciformes', 24.0),
                   ('mammal', 'Carnivora', 80.2),
                   ('mammal', 'Primates', np.nan),
                   ('mammal', 'Carnivora', 58)],
                  index=['falcon', 'parrot', 'lion', 'monkey', 'leopard'],
                  columns=('class', 'order', 'max_speed'))
grouped = df.groupby('class')
grouped = df.groupby('order', axis='columns')
grouped = df.groupby(['class', 'order'])

### Group by one or more columns

In [3]:
df = pd.DataFrame({
    'A': ['foo', 'bar', 'foo', 'bar', 'foo', 'bar', 'foo', 'foo'],
    'B': ['one', 'one', 'two', 'three', 'two', 'two', 'one', 'three'],
    'C':
    np.random.randn(8),
    'D':
    np.random.randn(8)
})
grouped = df.groupby('A')
grouped = df.groupby(['A', 'B'])

### Group by all but the specified columns

In [4]:
grouped.sum()

C         D
A   B                        
bar one    0.506752 -0.168305
    three -0.611635 -0.125627
    two   -1.659047 -0.348189
foo one    0.593322 -1.791624
    three -0.761311  1.573382
    two   -0.527922  1.293850

### Group by columns

In [5]:
def get_letter_type(letter):
    if letter.lower() in 'aeiou':
        return 'vowel'
    else:
        return 'consonant'

In [6]:
grouped = df.groupby(get_letter_type, axis=1)

### Group by index with duplicate values

In [7]:
lst = [1, 2, 3, 1, 2, 3]

In [8]:
s = pd.Series([1, 2, 3, 10, 20, 30], lst)

In [9]:
grouped = s.groupby(level=0)

In [10]:
grouped.first()
grouped.last()
grouped.sum()

1    11
2    22
3    33
dtype: int64

## Group by sorting

In [11]:
df2 = pd.DataFrame({'X': ['B', 'B', 'A', 'A'], 'Y': [1, 2, 3, 4]})

In [12]:
df2.groupby(['X']).sum()

,Y
X,
A,7
B,3


In [13]:
df2.groupby(['X'], sort=False).sum()

,Y
X,
B,3
A,7


## GroupBy with MultiIndex

In [14]:
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]

In [15]:
index = pd.MultiIndex.from_arrays(arrays, names=['first', 'second'])

In [16]:
s = pd.Series(np.random.randn(8), index=index)
grouped = s.groupby(level=0)
grouped.sum()
s.groupby(level='second').sum()
s.sum(level='second')
s.groupby(['first', 'second']).sum()

first  second
bar    one      -0.423939
       two       0.686123
baz    one      -0.221972
       two       0.935742
foo    one      -0.331134
       two      -0.287562
qux    one       0.900500
       two      -0.410583
dtype: float64

## DataFrame column selection in GroupBy

In [17]:
data = ['aaabb', 'xxyxy', range(5), range(1, 10, 2)]
columns = 'ABCD'
df = pd.DataFrame({name: list(value) for name, value in zip(columns, data)})
df[['A', 'C']].groupby('A').sum()
df.groupby('A')[['C']].sum()

,C
A,
a,3
b,7


## Iterating through groups

In [18]:
grouped = df[['A', 'C', 'D']].groupby('A')
for name, group in grouped:
    print(name)
    print(group)

a
   A  C  D
0  a  0  1
1  a  1  3
2  a  2  5
b
   A  C  D
3  b  3  7
4  b  4  9


## Selecting a group

In [19]:
grouped.get_group('a')

,A,C,D
0,a,0,1
1,a,1,3
2,a,2,5


## Aggregation

In [20]:
# ### Group by one or more columns
df.groupby('A').agg(np.sum)
df.groupby(['A', 'B']).agg(np.sum)
df.groupby(['A', 'B']).sum()

C  D
A B      
a x  1  4
  y  2  5
b x  3  7
  y  4  9

In [21]:
# ### Reset the index after grouping
df.groupby(['A', 'B']).agg(np.sum).reset_index()
df.groupby(['A', 'B'], as_index=False).agg(np.sum)

,A,B,C,D
0,a,x,1,4
1,a,y,2,5
2,b,x,3,7
3,b,y,4,9


### Get the size & description of each group

In [22]:
df.groupby(['A', 'B']).size()
df.groupby(['A', 'B']).describe()

C                                               D                      \
    count mean       std  min   25%  50%   75%  max count mean       std  min   
A B                                                                             
a x   2.0  0.5  0.707107  0.0  0.25  0.5  0.75  1.0   2.0  2.0  1.414214  1.0   
  y   1.0  2.0       NaN  2.0  2.00  2.0  2.00  2.0   1.0  5.0       NaN  5.0   
b x   1.0  3.0       NaN  3.0  3.00  3.0  3.00  3.0   1.0  7.0       NaN  7.0   
  y   1.0  4.0       NaN  4.0  4.00  4.0  4.00  4.0   1.0  9.0       NaN  9.0   

                         
     25%  50%  75%  max  
A B                      
a x  1.5  2.0  2.5  3.0  
  y  5.0  5.0  5.0  5.0  
b x  7.0  7.0  7.0  7.0  
  y  9.0  9.0  9.0  9.0

## Applying multiple functions at once

In [23]:
df[['A', 'C', 'D']].groupby('A').agg([np.sum, np.mean, np.std])

C                  D               
  sum mean       std sum mean       std
A                                      
a   3  1.0  1.000000   9    3  2.000000
b   7  3.5  0.707107  16    8  1.414214

## Named aggregation

In [24]:
animals = pd.DataFrame(
    {'kind': ['cat', 'dog', 'cat', 'dog'],
    'height': [9.1, 6.0, 9.5, 34.0],
    'weight': [7.9, 7.5, 9.9, 198.0]})

In [25]:
animals.groupby("kind").agg(
    min_height=pd.NamedAgg(column='height', aggfunc='min'),
    max_height=pd.NamedAgg(column='height', aggfunc='max'),
    average_weight=pd.NamedAgg(column='weight', aggfunc=np.mean))

,min_height,max_height,average_weight
kind,,,
cat,9.1,9.5,8.90
dog,6.0,34.0,102.75


In [26]:
# tuples can also be used instead of NamedAgg
animals.groupby("kind").agg(
    min_height=pd.NamedAgg(column='height', aggfunc='min'),
    max_height=pd.NamedAgg(column='height', aggfunc='max'),
    average_weight=pd.NamedAgg(column='weight', aggfunc=np.mean))

,min_height,max_height,average_weight
kind,,,
cat,9.1,9.5,8.90
dog,6.0,34.0,102.75


## Applying different functions to DataFrame columns

In [27]:
animals.groupby("kind").agg({
    'height': np.sum,
    'weight': lambda x: np.std(x, ddof=1)
    })

,height,weight
kind,,
cat,18.6,1.414214
dog,40.0,134.703842


strings can be used for the function name

In [28]:
animals.groupby("kind").agg({
    'height': 'sum',
    'weight': 'std'
    })

,height,weight
kind,,
cat,18.6,1.414214
dog,40.0,134.703842
